In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
from shapely.geometry import box
import supervision as sv
import torch
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import random
import seaborn as sns
from pathlib import Path
import matplotlib.patches as patches
from tqdm import tqdm

In [2]:
def get_iou_xyxy(box1, box2):
    bounding_box_1 = box(box1[0], box1[1], box1[2], box1[3])
    bounding_box_2 = box(box2[0], box2[1], box2[2], box2[3])

    area_of_intersection = bounding_box_1.intersection(bounding_box_2).area
    area_of_union = bounding_box_1.union(bounding_box_2).area

    iou = area_of_intersection / area_of_union if area_of_union != 0 else 0
    return iou

In [3]:
def get_11_pt_ap(precision, recalls):
    if len(precision) == 0 or len(recalls) == 0:
        return 0.0

    # Sort the pairs according to recall
    indices = np.argsort(recalls)
    sorted_recall = np.array(recalls[indices])
    sorted_precision = np.array(precision[indices])

    # 11-point interpolation
    ap = 0.0

    for t in np.linspace(0.0, 1.0, 11):
        p = np.max(sorted_precision[sorted_recall >= t]) if np.sum(sorted_recall >= t) > 0 else 0.0
        ap += p / 11.0

    return ap

In [4]:
def get_101_pt_ap(precisions, recalls):
    if len(precisions) == 0 or len(recalls) == 0:
        return 0.0

    # Sort the pairs according to recall
    indices = np.argsort(recalls)
    sorted_recall = np.array(recalls[indices])
    sorted_precision = np.array(precisions[indices])

    # 101-point interpolation
    ap = 0.0
    for t in np.linspace(0.0, 1.0, 101):
        p = np.max(sorted_precision[sorted_recall >= t]) if np.sum(sorted_recall >= t) > 0 else 0.0
        ap += p / 101.0

    return ap

In [5]:
def compute_ap_auc(precisions, recalls):
    if len(precisions) == 0 or len(recalls) == 0:
        return 0.0

    # Sort the pairs according to recall
    indices = np.argsort(recalls)
    sorted_recalls = np.array(recalls)[indices]
    sorted_precisions = np.array(precisions)[indices]

    # Make sure precision are in decreasing order
    for i in range(len(sorted_precisions) - 2, -1, -1):
        sorted_precisions[i] = max(sorted_precisions[i], sorted_precisions[i + 1])

    # Compute area under the curve using the trapezoidal rule
    ap = 0.0
    for i in range(len(sorted_recalls) - 1):
        dx = sorted_recalls[i + 1] - sorted_recalls[i]
        f_x = (sorted_precisions[i] + sorted_precisions[i + 1]) / 2.0
        ap += f_x * dx

    return ap

In [6]:
def generate_random_boxes(n_boxes, box_size, image_size,i):
    boxes = []
    for _ in range(n_boxes):
        # this is to ensure the box stays inside the image
        x1 = random.randint(0, image_size - box_size)
        y1 = random.randint(0, image_size - box_size)
        x2 = x1 + box_size
        y2 = y1 + box_size
        boxes.append([x1, y1, x2, y2,i])
    return boxes

In [22]:
def get_precision_recall(gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
    # Sort predictions by confidence score (highest first)
    sorted_indices = np.argsort(pred_scores)[::-1]
    sorted_pred_boxes = [pred_boxes[i] for i in sorted_indices]

    # Initialize variables
    n_gt = len(gt_boxes)
    tp = np.zeros(len(sorted_pred_boxes))
    fp = np.zeros(len(sorted_pred_boxes))

    # Evaluate each prediction
    for i, pred_box in enumerate(sorted_pred_boxes):
        # Find the corresponding ground truth box index
        gt_idx = sorted_indices[i]

        # Calculate IoU with the corresponding ground truth box
        iou = get_iou_xyxy(pred_box, gt_boxes[gt_idx])

        # Assign as true positive or false positive
        if iou >= iou_threshold:
            tp[i] = 1
        else:
            fp[i] = 1


    # Compute precision and recall
    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)

    precisions = tp_cumsum / (tp_cumsum + fp_cumsum)
    recalls = tp_cumsum / n_gt

    return precisions, recalls

In [7]:
def get_precision_recall_best_match(gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
    # Sort predictions by confidence score (highest first)
    sorted_indices = np.argsort(pred_scores)[::-1]
    sorted_pred_boxes = [pred_boxes[i] for i in sorted_indices]

    # Initialize variables
    n_gt = len(gt_boxes)
    tp = np.zeros(len(sorted_pred_boxes))
    fp = np.zeros(len(sorted_pred_boxes))
    # this is to ensure, each ground truth box is matched at most once
    gt_matched = [False] * n_gt

    # Evaluate each prediction
    for i, pred_box in enumerate(sorted_pred_boxes):
        # Find the best matching ground truth box
        max_iou = 0.0
        best_gt_idx = -1

        for j, gt_box in enumerate(gt_boxes):
            if gt_box[-1] == pred_box[-1]:
                if not gt_matched[j]:
                    iou = get_iou_xyxy(pred_box, gt_box)
                    if iou > max_iou:
                        max_iou = iou
                        best_gt_idx = j
            else:
                continue

        # Assign as true positive or false positive
        if max_iou >= iou_threshold and best_gt_idx != -1:
            tp[i] = 1
            gt_matched[best_gt_idx] = True
        else:
            fp[i] = 1

    # Compute precision and recall
    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)

    precisions = tp_cumsum / (tp_cumsum + fp_cumsum)
    recalls = tp_cumsum / n_gt

    return precisions, recalls

In [24]:
def compute_ap_metrics(total_gt_boxes, total_pred_boxes, total_pred_scores, iou_threshold=0.5):
    # Flatten all boxes and scores across images
    gt_boxes_flat = []
    pred_boxes_flat = []
    pred_scores_flat = []

    for i in range(len(total_gt_boxes)):
        gt_boxes_flat.extend(total_gt_boxes[i])
        pred_boxes_flat.extend(total_pred_boxes[i])
        pred_scores_flat.extend(total_pred_scores[i])

    # Compute precision and recall
    precisions, recalls = get_precision_recall(gt_boxes_flat, pred_boxes_flat, pred_scores_flat, iou_threshold)

    # Compute AP using three different methods
    ap_pascal = get_11_pt_ap(precisions, recalls)
    ap_coco = get_101_pt_ap(precisions, recalls)
    ap_auc = compute_ap_auc(precisions, recalls)

    return {
        "pascal_voc": ap_pascal,
        "coco": ap_coco,
        "auc": ap_auc
    }

In [8]:
def compute_ap_metrics_best_match(total_gt_boxes, total_pred_boxes, total_pred_scores, iou_threshold=0.5):
    
    gt_boxes_flat = []
    pred_boxes_flat = []
    pred_scores_flat = []

    for i in range(len(total_gt_boxes)):
        gt_boxes_flat.extend(total_gt_boxes[i])
        pred_boxes_flat.extend(total_pred_boxes[i])
        pred_scores_flat.extend(total_pred_scores[i])
    
    precisions, recalls = get_precision_recall_best_match(gt_boxes_flat, pred_boxes_flat, pred_scores_flat, iou_threshold)

    # Compute AP using three different methods
    ap_pascal = get_11_pt_ap(precisions, recalls)
    ap_coco = get_101_pt_ap(precisions, recalls)
    ap_auc = compute_ap_auc(precisions, recalls)

    return {
        "pascal_voc": ap_pascal,
        "coco": ap_coco,
        "auc": ap_auc
    }

In [9]:
def get_random_boxes(n_images=10, n_gt_boxes=10, n_pred_boxes=10,
                    box_size=20, image_size=100):
    total_gt_boxes = []
    total_pred_boxes = []
    total_pred_scores = []

    for i in range(n_images):
        # Generate ground truth boxes
        gt_boxes = generate_random_boxes(n_gt_boxes, box_size, image_size,i)
        total_gt_boxes.append(gt_boxes)

        # Generate predicted boxes
        pred_boxes = generate_random_boxes(n_pred_boxes, box_size, image_size,i)
        total_pred_boxes.append(pred_boxes)

        # Generate random scores for predictions
        pred_scores = [random.random() for j in range(n_pred_boxes)]
        total_pred_scores.append(pred_scores)

    return total_gt_boxes, total_pred_boxes, total_pred_scores

In [31]:
total_gt_boxes, total_pred_boxes, total_pred_scores = get_random_boxes()

ap_metrics = compute_ap_metrics(total_gt_boxes, total_pred_boxes, total_pred_scores, iou_threshold=0.5)
ap_metrics_best_match = compute_ap_metrics_best_match(total_gt_boxes, total_pred_boxes, total_pred_scores, iou_threshold=0.5)

# Print results
print("AP50 Results (Corresponding Match):")
print(f"Pascal VOC (11-point): {ap_metrics['pascal_voc']:.4f}")
print(f"COCO (101-point): {ap_metrics['coco']:.4f}")
print(f"Area Under PR Curve: {ap_metrics['auc']:.4f}")

print()

print("AP50 Results (Best Match):")
print(f"Pascal VOC (11-point): {ap_metrics_best_match['pascal_voc']:.4f}")
print(f"COCO (101-point): {ap_metrics_best_match['coco']:.4f}")
print(f"Area Under PR Curve: {ap_metrics_best_match['auc']:.4f}")


AP50 Results (Corresponding Match):
Pascal VOC (11-point): 0.0028
COCO (101-point): 0.0006
Area Under PR Curve: 0.0003

AP50 Results (Best Match):
Pascal VOC (11-point): 0.0546
COCO (101-point): 0.0218
Area Under PR Curve: 0.0172
